# Simple Machine Learning from Scratch with Python 3 and the IMDb:
## An Automatic Document Classification Adventure

## By Leigh Schumann

Machine learning is all the rage. If you don't know anything about machine learning (ML), my goal here is to get you started with a few ML methods and the math behind them. We're going to be doing this through the Natural Language Processing task of Automatic Document Classification. We'll be using the same data set through three methods for machine learning classification. In this tutorial, you'll learn about a bunch of different methods for ADC and implement them with Python. (If you don't know Python, I would recommend starting [here](introduction to python).) You can follow along by cutting and pasting into a `.py` document, or retyping the code, or downloading and executing the Jupyter Notebook from Github [LINK]. I recommend retyping, I find I learn more that way. Some knowledge of mathematical symbols is useful for this tutorial!

### Applications of ADC

Document classification is the practice of taking a text document and putting it into one of several categories. Automatic document classification (ADC) is the practice of teaching computers to do that. We're going to use the methods we cover here for ADC, but they can also be generalized to other kinds of classification tasks, not just document classification.

Automatic document classification can be profoundly useful and has many applications. What applications, you might ask? Well, if you use a browser like Chrome, and you visit a website in a language you don't understand, a little icon might pop up in the top right of the address bar that translates the page if you click it. Before it can show you that botton, the browser has to figure out what the language of the webpage is in the first place. It does this through automatic document classification.

![Image of the Chrome address bar with the translate button](img/address.png)

Some companies have email systems where if you email a question to the company as a whole, your email will automatically get forwarded to a particular person in the company who knows the answer to your question. It does this by automatically deciding which person knows the most about the topic of your question. In other words, it uses automatic document classification.

Your email spam filter? Automatic document classification. The system automatically puts the emails coming to your address into either spam or non-spam.

Say you started a company. Your company sells a new thing called a Torpax, which is super exciting and innovative. You want to see how people like Torpaxes. You could stand outside and ask lots of people, but that would take a long time and it's Winter so it's cold outside. What if you searched social media to see what people were saying about Torpaxes? That would also take a long time to do, unless...

You could use ADC! You could have your system take a look at all the social media documents mentioning Torpaxes and have it put it into the categories of positive or negative. That way, you could guage public opinion relatively quickly. You'll then be able to make decisions about how to market your Torpaxes.

This is usually called [sentiment analysis](https://towardsdatascience.com/sentiment-analysis-concept-analysis-and-applications-6c94d6f58c17), and is a very common application of ADC. In this tutorial, we're going to focus on sentiment analysis.

### The Data

We're going to be using a very popular sentiment analysis data set called the IMDb Dataset. You're probably familiar with the Internet Movie Database, and the reviews they have for lots and lots of movies. The IMDb Dataset is a collection of their reviews paired with a label of either positive or negative. Our task will be to use ADC to try to guess which of these categories each of these reviews belongs in.

![Image of the IMDb Logo](img/imdb.png)

In order to access the data, you're going to need to install NLTK, a natural language toolkit for Python. It comes with a lot of good language data and tools to process it. You can learn about how to install it [here](https://www.nltk.org/install.html). Luckily this is the only library you'll need to install for this tutorial.

A collection of texts/documents/language data is often called a corpus. The plural of corpus is corpora! (Kinda cool word, isn't it?) This terminology will be helpful later!

Ok, let's load the corpus. If you're running this for the first time, you probably going to need internet connection.

In [ ]:
# get the loader for the corpus
from nltk.corpus import movie_reviews

Ok, we'll be able to access the corpus through the `movie_reviews` variable. The next thing we want to do is partition the data into a training set and a test set. 

If you're unfamiliar with machine learning, you might not know those terms. The goal of most machine learning is to use data to train a model that can make accurate predictions about data it has never seen before. As you might guess, a training set is the set of data you use to train the model, the data that it can learn from and adjust itself to. In contrast, the test set is data that the model never ever uses for training. You put the test set aside at the beginning, and once training is over, you try to make predictions using your test set. If your predictions are good, you probably have a model that generalizes to brand new data pretty well!

It's not uncommon to develop a model that makes really good predictions about the training set but mediocre or bad predictions on the test set. It may be tempting to think that a model that does really well on the training set is a good model, but remember that in the real world, the model will always be used on brand new data it was not trained on. It can be a frustrating process to get better performance on unseen data, but keep at it and you can do amazing things!

So, let's get back to the code and make this split happen. For every document in an NLTK corpus, NLTK assigns it a file ID based on file names and locations. We'll use these file IDs to sort the corpus into training and test sets. We're going to use 75% of the total data for training, and 25% for testing.

In [ ]:
# let's get the list of all the file IDs
files = movie_reviews.fileids()

# we'll need to get a function from random to randomize which reviews are in either set.
from random import shuffle
shuffle(files)

# now let's find the index where we should make the split
split_idx = int(0.75 * len(files))

# and now let's make the split!
train_ids = files[:split_idx]
test_ids = files[split_idx:]

Ok, so we have our train and test sets. Let's take a moment to check out what the data actually look like. Let's look at a random review and it's label.

In [ ]:
# You can check out a different review every time you run this, with the help of our good friend randint()
from random import randint

idx = randint(0, len(train_ids)-1)
fileid = train_ids[idx]
# get the raw data corresponding to this file ID
review = movie_reviews.raw(fileid)
# categories() returns a list of categories, but since there's only one, pos or neg, 
# we'll get the zeroth one
label = movie_reviews.categories(fileid)[0]

print(review, "\n-----------------------\nLabel:", label)

So, would you want to watch that movie? Now we need to do some more important work with our data, called preprocessing.

### Preprocessing

Preprocessing is the way you make your raw data usable for the model or more easy for the model to understand. Preprocessing may require some calculations, which you'll see below, and it's important to remember to only do these calculations on the training set, even though you'll need to preprocess the test set as well. If this is a bit confusing, it will become clearer by the end of this section.

First we're going to grab all the documents in the corpus as a list of lists of words for the train and test sets. We'll use the list of file IDs we created earlier to get these.

In [ ]:
train = [movie_reviews.words(fileid) for fileid in train_ids]
test = [movie_reviews.words(fileid) for fileid in test_ids]

In addition to a list of all the reviews, we need a list of all the labels of those reviews, positive and negative. When it comes time to use the labels in training or check our predictions against the actual labels, we'll look it up in the list we're about to create.

In [ ]:
# The categories() method returns a list of the categories of a document.
# There is only one category for each document, but it always returns a list to support NLTK corpora
# which have more than one label per document. So we'll index at 0 the list that it returns.
train_labels = [movie_reviews.categories(fileid)[0] for fileid in train_ids]
test_labels = [movie_reviews.categories(fileid)[0] for fileid in test_ids]

The first real preprocessing step we're going to do is called stemming. Stemming is the process of taking words and returning their stems. For the English language, this usually means taking off suffixes if present. For instance, the word "going" would be stemmed to "go", "cars" would be stemmed to "car", etc.

This step is based on the assumption that the model doesn't need to consider "going" and "go" as different words in order to predict whether the review is positive or negative. In fact, because they are so similar, we can assume that they are the same.

NLTK comes with several stemmers. We'll be using the NLTK snowball stemmer. The stemmer we use will also turn all the letters lowercase. This is based on the assumption that the model doesn't need to think that words like "Good" and "good" are different.

The stemmer may have some surprising results. For instance, it likes to take the letter "e" off the end of words. This is makes things simpler in the long run, because it's much much easier to go from "surprise" to "surpris" and "surprising" to "surpris" than figure out that you need to both take off the "ing" and add an "e" in the case of "surprising".

This step may take a while!

In [ ]:
from nltk.stem.snowball import EnglishStemmer

stemmer = EnglishStemmer()

train = [[stemmer.stem(word) for word in review] for review in train]
test = [[stemmer.stem(word) for word in review] for review in test]

Our next step will be to cut out uncommon words. We want to do this because if a word appears only one or two times in our training set, it could be very misleading for the model. What if the word "blue" just randomly happens to appear in two negative reviews? The model may take the word "blue" to be negative even though it was just random chance that it showed up in negative reviews. Cutting out uncommon words lowers the probability of this happening.

This requires some counting and calculation. This is where that note about calculating using the training set comes in! We can only count up the frequencies of the words in the training set, because of course we don't know what word frequencies will be like in data we have never seen before, and the test set is meant to be data we have never seen before.

We're going to use the most common 2,000 words! Here we go.

In [ ]:
# we'll import a class from collections which will help us in our task
from collections import Counter

# WE CAN ONLY USE THE TRAIN SET FOR THIS PART!!
words = [word for review in train 
              for word in review]

word_counts = Counter(words)

top_words = word_counts.most_common(2000)
# we don't really need the counts, so we'll take those out
top_words = [item[0] for item in top_words]
# and we'll turn it into a set which will let us search it faster
top_words = set(top_words)

Ok, now we've got our top 2,000 words. Now we're going to make it so that all the words that are not in that top 2,000 are replaced by a stand-in which we will call UNK for unknown.

In [ ]:
def replace_uncommon(data):
    for review in data:
        for i, word in enumerate(review):
            if word not in top_words:
                # replace the word at that index with the UNK token
                review[i] = 'UNK'


replace_uncommon(train)
replace_uncommon(test)

Ok, that's all the preprocessing we'll do for now! Let's take a look at what our preprocessed data looks like now! We'll look at the same review we saw earlier.

In [ ]:
fileid = train_ids[idx]
# let's get the preprocessed review
preped_review = " ".join(train[idx])  # since it's a list, we need to join it to make it a string
# categories() returns a list of categories, but since there's only one, pos or neg, 
# we'll get the zeroth one
label = movie_reviews.categories(fileid)[0]

print(preped_review, "\n-----------------------\nLabel:", label)

Now we'll start our first Automatic Document Classification method!! HURAAAAYYYY!!!!!!

### K Nearest Neighbors

K Nearest Neighbors, or kNN, is technically not a machine learning method. It generally doesn't perform very well and is very rarely used in industry. Why are we looking at it? It's very useful as a baseline. It's intuitive and simple, and if your super sophisticated machine learning model doesn't perform better than kNN, you know something is wrong.

So what is kNN? 

To do kNN, you need three things: data, a whole number `k`, and a distance function `dist(a, b)` which will return the distance between different data points. To make a prediction about about a data point in the test set, you simply look through the training set, getting the distance from the data point to each of the data points in the training set. You then look at the `k` closest points in the training set and their labels. The most common label among those `k` neighbors will be the label you predict.

See? Super simple. No hard math involved. kNN is not considered a machine learning method for a variety of reasons. One obvious reason is that it doesn't really learn. It just uses the training data directly without ever looking at it before testing. It also doesn't leave behind an "artifact." There's no explicit model that it produces that is seperate from the training data itself, which most machine learning methods do produce. But, as I said above, it is useful as a baseline and a first step for learning about machine learning.

So the first thing we'll do is set `k`. Seven is a good low number that isn't too low, and is odd, so it will never come up with a tie since we have only two categories.

In [ ]:
k = 7

Now let's define our distance function. We're going to convert each of our reviews to sets of words, so there are no duplicate words, and our distance function will be the Jaccard Distance. This metric is just 1 - (the Jaccard Index). So let's look at the formula for the Jaccard Index:

$$
J(A,B) = \frac{A \cap B}{A \cup B}
$$

The Jaccard Index is just the cardinality (or size) of the [intersection](https://brilliant.org/wiki/sets-union-and-intersection-easy/) of the two sets divided by the cardinality of the [union](https://brilliant.org/wiki/sets-union-and-intersection-easy/) of the two sets. So the Jaccard Distance is just:

$$
D(A,B) = 1 - J(A,B) = 1 - \frac{A \cap B}{A \cup B}
$$

You can read more about the Jaccard Index and Jaccard Distance [here](https://www.statisticshowto.datasciencecentral.com/jaccard-index/).

Ok, let's implement that distance function in Python:

In [ ]:
def dist(a, b):
    a, b = set(a), set(b)
    # handily, in Python you can use & to get the intersection
    inter = a & b
    # and the pipe | to get the union
    union = a | b
    # Now we get the Jaccard Index
    index = len(inter)/len(union)
    # and we'll return the distance
    return 1 - index

Wow, great, we have our data, we have `k`, we have `dist(a, b)`, let's make some predictions about our test set!

In [ ]:
def knn_predict(x):
    """
    Predicts the label of a data point using the kNN method
    :param x: a data point from the test set
    :return: the predicted label
    """
    # this list will contain tuples of the distance to a neighbor and the label of that neighbor
    # for all data points in the training set
    all_neighbors = []
    # we need the distances to all the data points in the training set.
    # This is a long process.
    for i, neighbor in enumerate(train):
        distance = dist(x, neighbor)
        # the indices of the lists match up so it's no problem to get the label of the neighbor.
        neigh_label = train_labels[i]
        # now append the tuple of those two!
        all_neighbors.append( (distance, neigh_label) )
    # We have the list, now sort it to find the closest ones.
    all_neighbors.sort()
    # And we get the top k nearest neighbors!
    nearest_neighbors = all_neighbors[:k]
    nearest_labels = [label for (distance, label) in nearest_neighbors]
    # now we just need to find the most common label in those nearest neighbors
    # we'll use Counter class again.
    counted_labels = Counter(nearest_labels)
    # most_common() returns a list of tuples, so we need to access the zeroth entry of the only tuple.
    prediction = counted_labels.most_common(1)[0][0]
    # And we're done!
    return prediction

So now we have a function that makes a prediction about a single data point in the test set. Let's take a look at what it predicts for an individual example.

In [ ]:
def print_random_prediction(predict, test_data):
    """
    Predicts a random instance from the passed-in data, and prints that review's text as well as the
    prediction and actual label.
    :param predict: the function which makes a prediction about instances
    :param test_data: the test data from which we will make a random selection
    """
    idx = randint(0, len(test_ids)-1)
    fileid = test_ids[idx]
    # get the raw data corresponding to this file ID
    review = movie_reviews.raw(fileid)
    # and let's get the preprocessed review
    preped_review = " ".join(test[idx])  # since it's a list, we need to join it to make it a string
    # categories() returns a list of categories, but since there's only one, pos or neg, 
    # we'll get the zeroth one
    label = movie_reviews.categories(fileid)[0]
    
    print(review, 
          "\n-----------------------\n", preped_review,
          "\n-----------------------\nPredicted Label:", predict(test_data[idx]),
          "\nActual Label:", label)


print_random_prediction(knn_predict, test)

How interesting! Did it predict correctly? Would you have predicted the same thing as kNN?

Now let's make predictions for the entire test set! The big moment has arrived. 

We're going to make a function that we can use as the evaluation function for all of our different ADC methods, and then we'll use it for kNN and have a look at the results! Our evaluation function will just predict the entire test set and then divide the number we got correct by the total size of the test set. That's called our accuracy score.

The prediction process will probably take quite a while. The algorithmic complexity of making a single prediction in kNN is at least $\text{O}(n)$, where $n$ is the number of instances in the training set. If you don't know what that means and would like to learn more about algorithmic complexity, take a look [here](https://discrete.gr/complexity/). The algorithm's relative ineffeciency is another weakness of kNN.

In [ ]:
def evaluate(predict, data):
    """
    Prints the score that a certain prediction method attains on a given data set
    :param predict: the function that does the predicting. This will be different for different ADC methods.
    :param data: the data for the prediction method to predict. Usually the test set.
    """
    # Let's keep track of the number of correct predictions
    num_correct = 0
    # And now let's iterate through the test set!
    for i, x in enumerate(data):
        # first we get a prediction for the data point
        prediction = predict(x)
        # and then we get its actual label from the list. Remember the indices will line up!
        actual = test_labels[i]
        if prediction == actual:
            num_correct += 1
    # now the fraction that's correct will just equal the number correct over the length of the test set
    frac_correct = num_correct / len(data)
    # and we'll print the score as a percent!
    perc_correct = frac_correct * 100
    print("%.2f%%" % perc_correct)


# Ok, now let's evaluate kNN!
evaluate(knn_predict, test)

The score you get might be different from the one I get, mainly because of differences in the way our computers shuffled the data way up at the beginning.

I got 73.40%! That's really pretty good! Since there are only 2 different labels, if we guessed at random, we would expect the score to be about 50%. So this is a 46.8% improvement over random chance! That's cool! How much better is your score than random chance? You can find out by calculating

$$
\text{percent improvement} = 100 * \frac{ \text{score} - \text{chance} }{ \text{chance} }
$$

So you might see why kNN makes a good baseline. It is easy to understand and implement, and it tends to obtain a better score than chance. However, it has major drawbacks. I mentioned above that it's quite slow, and that, technically, kNN isn't really machine learning. 

Another major drawback is that every word that appears in the review is weighted equally. In reality, if we're trying to predict the opinion of the reviewer, a word like "bad" would probably be much much more important than a word like "and". One method that addresses this issue is Naive Bayes. Here we go!!

### Naive Bayes

Naive Bayes is a classification method derived from Bayes' Theorem. If you're unfamiliar with Bayes' Theorem and its implications, Veritasium made a great [video](https://www.youtube.com/watch?v=R13BD8qKeTg) about it. Take a look at it if you don't know what the theorem is or what it does!

So now I'm assuming you've either seen the video or you know about Bayes' Theorem. Now let's dig into the math and talk about how you go from the theorem to Naive Bayes. Bayes' Theorem reads

$$
\text{P}(l | w) = \frac{ \text{P}(w | l) \text{P}(l) }{\text{P}(w)}
$$

Where $l$ is a label of the data, in our case positive or negative, and $w$ is an observation, in our case a word.  If we put Bayes' Theorem in English, it might look like this:

$$
\text{probability of a hypothesis given an observation} = \frac{ (\text{probability making this observation given that the hypothesis is true}) * (\text{probability of this hypothesis in general}) }{ \text{probability of making this observation in general} }
$$

We can't can't make the above calculation if we treat the entire document as a single observation. Documents almost never recur, making frequency and probability calculations complicated. We are going to make calculations by looking at individual words. We treat each word as a new observation, updating our probability as we go, eventually calculating the probability that a given document belongs to a given label.

So we have the Bayes, but where does the "Naive" in "Naive Bayes" come from? We assume that every observation (i.e. each word of the text) is independent and unrelated. This is probably not true, but it simplifies things greatly and still yields pretty good results! There are methods that use Bayesian statistics that do not make this assumption, but they are less common in document classification and lie outside the scope of this tutorial! All we need to do is multiply together a modified version Bayes' theorem for each word in the document for each possible label.

Now, we aren't really interested in calculating the exact probablities of each label for each data point, we are just trying to find out which one is more likely for each data point. This leads us to two modifications. One is that we don't need to divide by the probability of the observation every time, because the probability of the observation is the same regardless of which hypothesis, positive or negative, is true. The second is that we can do the calculations with the log of the probabilities. This makes it easier for the computer to calculate, because otherwise the numbers get way too small!

So, now that you hopefully have an understanding of Naive Bayes, let's get to the implementation. The first thing we will need to do is train the model. This means we need to find two things: the probability of both labels, and the probability that each word would appear in each of those labels. 

First we'll find the probability of each of the labels.


In [ ]:
# we'll use our old friend the Counter class
counted_labels = Counter(train_labels)
all_labels = list(counted_labels.keys())
# now let's get the number of each of the labels present
label_probs = {}  # this dictionary will contain the probabilities of the labels
for label in all_labels:
    num_label = counted_labels[label]
    prob_of_label = num_label / len(train_labels)
    label_probs[label] = prob_of_label

Alright, that's our prior label probabilities!

Now we have to get the probability that we would see each word given each label. It might be tempting to use the empirical probability (or relative frequency), which would be:

$$
\frac{ \text{number of times a word appears in texts of a given class} }{ \text{number of all words that appear in that class} }
$$

However, this is never what you should do in practice. Imagine you never see the word "red" in negative reviews in your data. The empirical probability of seeing "red" in a positive review would be $\frac{0}{N} = 0$, where $N$ is the number of words found in all positive reviews. If you just used the empirical probability in your classifier, then as soon as you encountered the word "red" and multiply it's probability in, the probability of the review being positive drops to 0. Does this seem realistic to you? Probably not.

The way to address this problem is with smoothing. We'll use the add-one version of Laplace smoothing for our classifier, which makes our smoothed probability equal to

$$
\frac{ \text{number of times a word appears in texts of a given class } + 1 }{ \text{number of all words that appear in that class} + \text{number of unique words} }
$$

This "smoothes out" our empirical probabilities so that we don't make unwarrented assumptions like the word "red" cannot appear in a positive review.

Ok, so let's calculate these word probabilities!

In [ ]:
# first thing we want is to make a copy of top_words that includes the UNK token
all_words = set(top_words)
all_words.add('UNK')
# now we'll want to do is get a list of all the words that appear in all the review for each label
words = {}
# for each of our two labels
for label in all_labels:
    # we'll need a list to store all the words in review of that label
    words_in_label = []
    # and we'll go through every review
    for i, review in enumerate(train):
        if train_labels[i] == label:
            # add all the words of that review if it has the appropriate label
            words_in_label.extend(review)
    # and put that list of words into the list that contains the lists for both labels
    words[label] = words_in_label

# now let's make the dictionary that will store all the probabilities for the words!
word_probs = {label: {} for label in all_labels}

# Now we'll use our old friend the Counter class!!
for label in words.keys():
    words_in_label = words[label]
    counted = Counter(words_in_label)
    for word in all_words:
        if word in counted.keys():
            freq = counted[word]
        else:
            freq = 0
        smoothed_prob = (freq + 1) / (len(words_in_label) + len(all_words))
        word_probs[label][word] = smoothed_prob

Ok, that should do it for our training! Let's make a function that predicts a single data point. It will need to multiply up our Naive Bayes formula for each word for each label. In mathematical notation, our predicted label would be:

$$
\DeclareMathOperator*{\argmax}{arg\,max}
\text{predicted label } = \argmax_{l\in L}\Big(\, \sum_{w \in D} \log(\text{P}(w|l))+\log(\text{P}(l)) \,\Big)
$$

Where $L$ is the set of all labels, in our case positive or negative, and $D$ is the words in the document.

Here we go, let's implement that!


In [ ]:
from math import log


def bayes_predict(x):
    """
    Predicts the label of a data point using a Naive Bayes classifier
    :param x: the data point to be predicted
    :return: the predicted label
    """
    # ok let's make a dict that will store the probs of the two labels as we calculate them
    # the starting values will be the log of the prior probability of the label, which we calculated earlier
    calculated_probs = {label: log(prob) for label, prob in label_probs.items()}
    # now let's go through all the words of the review!
    for label in all_labels:
        for word in x:
            calculated_probs[label] += log(word_probs[label][word])
    
    # all we need to do is return the most likely one!
    return max(calculated_probs, key=calculated_probs.get)

There we have it! Let's see how Naive Bayes predicts a single random review, like we did for kNN! Isn't it nice that we already have a great function for that?

In [ ]:
print_random_prediction(bayes_predict, test)

Did it get it right? Run it a couple times and see what the results are. Does it seem to be getting things right more or less often than kNN?

Well, we can address the above question in a move empirical way by evaluating on the test set! Here we go! Once again, we already have the function done!

In [ ]:
evaluate(bayes_predict, test)

Interesting! I got 78.40%! That's a 56.8% improvement over chance and a 6.8% improvement over kNN. That may not sound like a lot, but that 6.8% could make a huge difference in industrial applications!

Notice that the evaluation was probably much quicker than it was for kNN. Not only did it do better than kNN, it was a lot more efficient! These two reasons are why Naive Bayes is basically always preferable to kNN when you're doing Automatic Document Classification, and why Naive Bayes sees a fair amount of use in industry while kNN sees basically none.

Alright, that was pretty cool, huh? Are you ready for our next method??

### Decision Trees

So far both of the algorithms we've looked at make independence assumptions. In both methods, if a review contains the word "good" and the word "bad", the model would consider it similarly to the reviews that contain just the word "good" and just the word "bad". But maybe that's not right... maybe if a review has both those words, it's something else that isn't just the sum of those parts...

Decision trees are one method that addresses this. Decision tree algorithms build a branching tree structure with a choice at each branch. When you classify a data point with this tree, you follow the branches that apply to that data point. Eventually you arrive at a leaf and there you have your predicted label.

![Image of a Simple Decision Tree](img/dec-tree.jpg)

Let's discuss the anatomy of complete decision trees. Let's start from the beginning. A tree is a type of graph, which means that it's a structure made up of nodes and the connections, or edges, between those nodes. Trees are hierarchically organized, with one node all the way at the top called the root. The nodes connected below another node $n$ are called node $n$'s children, and the one node connected above node $n$ is called node $n$'s parent. Each node that has no children is called a leaf.

In a decision tree, every non-leaf node has an attribute. In our case, that means it would have a word. Then, the edges to each of the children represent a different value for that attribute. In our case, we would have two values for each attribute: whether the word is present in the review or not. The leaves of the decision tree are given a label. As discussed above, to classify a review, you would start at the root, look at the attribute in that root, follow the edge that has the appropriate value, look at the attribute in that child, follow the edge that has the appropriate value, and so on until you get to a leaf, which has the predicted label! And it's as easy as that!

So we'll need a class which represents will represent the nodes of this tree, with which we can build the decision tree. Here's an implementation of what we just discussed: 

In [ ]:
class Node:
    """
    A node class to represent the nodes in a decision tree
    """

    def __init__(self, attribute=None, leaf_label=None):
        """
        :param attribute: the attribute that the node splits on
        :param all_values: attributes mapped to their values from the training set
        :param leaf_lable: if this is a leaf, pass me a leaf label!
        """
        # even though these two are mutually exclusive, I'm keeping them separate for readability
        self.attribute = attribute
        self.leaf_label = leaf_label

        # let's create the children.
        # If we were passed an attribute, then we will be having children.
        if self.attribute is not None:
            self.children = {True: None, False: None}
        # If we weren't passed an attribute, then it's a leaf and we won't have children.
        else:
            self.children = None
            
    
    def __str__(self):
        """
        Will help for printing! Very simple __str__() function
        """
        return self.attribute


    def is_leaf(self):
        """
        :return: whether or not the node is a leaf
        """
        return self.leaf_label is not None


Decision tree algorithms are nice for many reasons, one of them being that it's usually pretty easy for humans to understand the final resulting tree, giving people are good sense of what the algorithm learned. This contrasts with a method like artificial neural networks, which are usually quite difficult to interpret. It often requires quite sophisticated methods to understand what neural networks learned, compared to just going through the branches of the tree to understand decision trees.

#### Preprocessing for Decision Trees

To make our decision tree alrogithm easier to implement, we're going to do a little more preprocessing! Every data point needs to have exactly the same series of attributes. The "why" of this should become increasingly clear as we look at the algorithm. The way we'll do this is make all the possible words into all the attributes. We'll make each data point into a dictionary, mapping each word to `True` or `False`, `True` meaning the review contains the word, `False` meaning it doesn't contain it. Another thing we'll do to make it more workable this time around is to add the label of the review that data point dictionary, rather than keeping it in a seperate list like before.

In [ ]:
def dictify(data, labels):
    # This will be the list which will hold the data point dictionaries
    dicts = []
    # now let's iterate through train and make our dicts
    for i, review in enumerate(data):
        # here's the basic data point dict
        x = {w: False for w in all_words}
        # then we'll change all the ones that appear to True
        for word in review:
            x[word] = True
        # and now we'll add the label.
        # We can use an ALL CAPS key, because remember all words (except UNK) will be lower case
        x['LABEL'] = labels[i]
        # and now append it to our dicts list
        dicts.append(x)
    # return our completed list of dicts!
    return dicts

train_dicts = dictify(train, train_labels)
test_dicts = dictify(test, test_labels)

#### The Best Attribute

There are many algorithms for training/creating decision trees, and one prominent algorithm is called ID3, which stands for Iterative Dichotomizer 3. We'll be using ID3 to make our decision tree. Let's take a look at some pseudocode for ID3, adapted slightly from [Wikipedia](https://en.wikipedia.org/wiki/ID3_algorithm#Pseudocode), and then we'll talk about it.

```
ID3 (Examples, Attributes)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = "pos".
    If all examples are negative, Return the single-node tree Root, with label = "neg".
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the label in the examples.
    Otherwise Begin
        a ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = a.
        For each possible value, vi, of a,
            Add a new tree branch below Root, corresponding to the test a = vi.
            Let Examples(vi) be the subset of examples that have the value vi for a
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {a})
    End
    Return Root
```

You may notice two things about this algorithm. One is that it is recursive. The ID3 function described will repeatedly call itself in order to build the decision tree. Another thing you might notice is that the pseudocode doesn't define what "the Attribute that best classifies examples" actually means. We want to start with the best-classifying attribute and work our way down, but how do you determine which attribute is "best"?

Well, one popular way is to use Information Gain. As the name suggests, Information Gain measures how much information you would gain by using this attribute to split. Sounds like a nice metric, huh? Let's explore it.

To understand Information Gain, you first have to understand Entropy. In this situation, Entropy can be thought of as a measure of the "impurity" of the data. 

![An Image Showing Different Levels of Entropy and "Purity"](img/entropy-and-purity.png) 

In the whole of our training set, it should be that the reviews are around 50% positive and around 50% negative. This is highly "impure," the Entropy of the data set is high. If we picked a review at random, we would be just as surprised if that review turned out to be positive or if it turned out to be negative. If we just take out all the positive reviews, we would have an entirely "pure" subset, with an Entropy of 0. We would never be surprised to see a positive review if we picked at random, because we already know that the entire dataset is positive reviews! Entropy, that measure of "impurity," is defined for our purposes as

$$
H(T) = - \sum_{i=1}^n p(l_i)\log{p(l_i)}
$$

where $T$ is the set of the training instances we're looking at, $p(l_i)$ is the probability of a certain label. In this case, the set of all values is just the labels positive and negative. So you'll just sum that up, flip the sign, and you've got your Entropy!

Let's implement a function which returns the entropy of a given data set, and then we can move on to our discussion of information gain.

In [ ]:
def entropy(data):
    """
    Returns the entropy of a given attribute, that is, a measure of how heterogenous a data set is with
    respect to a certain attribute/variable. The higher the number, the more heterogenous.
    :param data: the data point dicts of the data set for which we want the entropy calculation
    :return: a calculation of the entropy
    """
    # if there are no data points, then we've gotten a subset of a data set that did not have a certain value in it.
    # we don't need to worry about it, and we'll just return 0.
    if len(data) == 0:
        return 0
    # count up all the instances with each label
    counts = {label: 0 for label in all_labels}
    for review in data:
        label = review['LABEL']
        counts[label] += 1
    # then convert it into proportions
    proportions = [count / len(data) for count in counts.values()]
    # multiply each proportion by its own log
    sum_this = [prop*log(prop) for prop in proportions if prop != 0]
    # then sum it all up and flip its sign and we're done!
    return -sum(sum_this)

Ok, so we have our entropy function! Yay! Now how about Information Gain? Basically, it's a measure of how much information you gain from splitting on a certain attribute of the training data, in our case, a certain word. That isn't too surprising, is it? Let's take a look at the formula and then we'll discuss what it means in our case.

$$
IG(T \mid a) = H(T) - \sum_{v\in vals(a)} \frac{\vert S_a(v) \vert}{\vert T \vert} * H(S_a(v))
$$

We're interested in $IG(T \mid a)$, which is the information gain of the splitting the training set/set of training examples we're looking at on the basis of attribute $a$. $H(T)$ is the entropy of the training set as we discussed above. 

Now how about the sum term on the right? That's where the magic is. The set $S_a(v)$ is the subset of $T$ in which all members have the value $v$ as the value of their attribute $a$. In other words 

$$
S_a(v) = \{ \textbf{x} \in T \mid x_a = v \}
$$

Let's write a function that returns $S_a(v)$:

In [ ]:
def subset(data, attribute, value):
    """
    Returns the subset of the given data which has a certain value for a certain attribute
    :param data: all the data we're working with at this time
    :param attribute: the attribute we're looking at
    :param value: the value we're looking for
    :return: a list of instances which have that attribute equal to that value
    """
    S_av = []
    for instance in data:
        if instance[attribute] == value:
            S_av.append(instance)
    return S_av

Great, we have that now! Let's take a look back at the Information Gain function.

$$
IG(T \mid a) = H(T) - \sum_{v\in vals(a)} \frac{\vert S_a(v) \vert}{\vert T \vert} * H(S_a(v))
$$

Remember that $\vert S \vert$ is the cardinality or size of any set $S$. So the fraction in the center of our equation is the ratio between the size of the subset of all instances where $x_a = v$ and the entire set. You then multiply this by the entropy of $S_a(v)$. This product represents the entropy of $S_a(v)$ weighted by the relative size of that set. If we sum all those products together, we get a representation of how entropic or "impure" the results will be if we split on attribute $a$. Then, when we subtract that sum from the entropy of the entire set, $H(T)$, we get our information gain. If the we start out with a quite entropic set $T$, and then splitting on $a$ produces two relatively large, relatively low-entropy sets, $IG(T \mid a)$ will be relatively large and $a$ will be a highly prefereable attribute to split on.

Say we're looking at a possible split on whether or not the review contains the word "finally". We start with a set of 100 training instances which have an entropy of around 0.998. Suppose that 66% of the reviews with the word "finally" are positive, and only 3 of 100 reviews contain "finally". This subset would have an entropy of around 0.918. The 97 of 100 reviews that do not contain "finally" are more entropic, fairly evenly split between positive and negative. It might have an entropy of around 0.999. Splitting on the word "finally" would probably not be preferred, because the low entropy subset it results in the creation of one small, low-weighted low entropy subset and one large, high-weighted high entropy subset.

On the other hand, suppose we start with that same set of 100 with an entropy of 0.998. Say we look at a possible split on whether or not the review contains the word "bad". Now suppose that 25 of those 100 reviews contain the word bad, and 80% of those 25 reviews are negative. This subset would have an entropy of around 0.722. The subset that doesn't contain "bad" is more entropic, with around 40% being negative and the rest of course being positive. This subset would have an entropy of around 0.970.

Which of these two options would have a higher information gain?

The answer is the second one! The first one has one subset with a somewhat low entropy, but it's quite small and gets weighted down. Meanwhile, the second has a subset that's pretty low entropy, and is relatively big, so it doesn't get too greatly weighted down.

So, let's write our information gain function! 

In [ ]:
def information_gain(data, attribute):
    """
    Calculates how much information will be gained by splitting on a given attribute.
    Helps us calculate the best attribute to split on.
    :param data: all the data that we're working with at this point
    :param attribute: the attribute that we're calculating information gain on
    :return: info gain, a measure of the amount of information that will be gained by splitting on this attribute
    """
    # get the entropy. We'll be using that.
    ent = entropy(data)
    # iterate through the values and gather up all the entropies for 
    # the subset with a given value times the proportion with that value
    sum_this = []
    for value in (True, False):  # for each attribute, i.e. each word, the possible values are True and False
        S_av = subset(data, attribute, value)
        
        prop = len(S_av) / len(data)
        ent_v = entropy(S_av)
        sum_this.append(prop*ent_v)
    # subtract the sum of those from the entropy to get our information gain!
    infogain = ent - sum(sum_this)

    return infogain

Great! That's our Information Gain. We now need a function which will return the attribute which gives us the highest information gain. We'll call it `argmax_infogain(data)`:

In [ ]:
def argmax_infogain(data):
    """
    Finds the best attribute to split on by finding the attribute that yields the highest information gain
    :param data: all the data that we're working with at this point
    :return: the best attribute to split on
    """
    # make a dict that will map attributes to the information gain on splitting them
    attributes_with_gain = {}
    # go through all the attributes that we could possibly split on
    for attribute in all_words:
        attributes_with_gain[attribute] = information_gain(data, attribute)
        
    best_attribute = max(attributes_with_gain, key=attributes_with_gain.get)
    return best_attribute

There we have it! And that's our precise definition of what the best attribute is. Now we can move on to the training!

#### Decision Tree Training

Let's take a look at our pseudocode for ID3 again.

```
ID3 (Examples, Attributes)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = "pos".
    If all examples are negative, Return the single-node tree Root, with label = "neg".
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the label in the examples.
    Otherwise Begin
        a ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = a.
        For each possible value, vi, of a,
            Add a new tree branch below Root, corresponding to the test a = vi.
            Let Examples(vi) be the subset of examples that have the value vi for a
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {a})
    End
    Return Root
```

Note the recusive call in the pseudocode:

```
            Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {a})
```

The object `Examples(vi)` is equivalent to $S_a(v)$ in the mathematical notation we used above. This means that every time we get further in building our tree, that is, whenever we make the recursive call, we are looking at a smaller and smaller sample of the data. Since we've been splitting on the "best attribute" as defined above, this also means that the samples we're looking at are getting "purer" and "purer" with each recursive call. This recursive process continues until either the remaining sample is purely one label or the other, or there is nothing left in the sample, in which case we just label it with the most common label in the data.

Now that we have all this understanding and a definition of the best attribute, can we implement the rest of the pseudocode? I think so! Here it is:

In [ ]:
def id3_train(data, attributes):
    """
    A recursive function which trains a decision tree based on the ID3 algorithm
    :param data: the data set we're currently looking at. This will get progressively smaller as we dichotomize.
    :param all_values: a dict mapping attributes to all their values. This will also get progressively smaller.
    :return: the trained tree
    """
    # let's create that root of the tree
    root = Node()
    # let's get a list of the labels we're working with
    labels = [point['LABEL'] for point in data]
    # if all instances are entirely one or the other label, we'll return a leaf node with that label
    if all(label == 'pos' for label in labels):
        root.leaf_label = 'pos'
        return root
    elif all(label == 'neg' for label in labels):
        root.leaf_label = 'neg'
        return root
    # we need to get the most common label in the data, which will be useful in several situations
    most_common_label = Counter(labels).most_common(1)[0][0]
    if len(attributes) == 0:
        root.leaf_label == most_common_label
        return root
    # If none of the above basecases are true, let's work on the recursive case.
    else:
        # find the best attribute
        a = argmax_infogain(data)
        # use it to make a node
        node = Node(attribute=a)
        # make a deep copy of the dict of attributes and values and delete the attribute just used from the dict
        less_attributes = list(attributes)
        less_attributes.remove(a)
        # train on the remainder for each possible value of this node's attribute:
        # True for contains that word, False for does not contain
        for value in (True, False):
            S_av = subset(data, a, value)
            # (if there's no more data with that value, we'll just give that node the most common label)
            if len(S_av) == 0:
                node.children[value] = Node(leaf_label=most_common_S_av)
            else:
                node.children[value] = id3_train(S_av, less_attributes)
        return node
    
decision_tree = id3_train(train_dicts, all_words)

Note that the training process may take your computer a long time!

#### Prediction and Evaluation

Great! We have trained our decision tree! We now need a way of getting predictions from that tree! We discussed traversal quite a bit above, but to reiterate, to classify a review, you would start at the root, look at the attribute in that root, follow the edge that has the appropriate value, look at the attribute in that child, follow the edge that has the appropriate value, and so on until you get to a leaf, which has the predicted label.

Let's implement that in Python! Note that what is described above is a recursive process, and we'll implement it as such in `tree_predict_recu(node, instance)`. However, we'll also have a wrapper function that can start the process from the beginning, which will be called `tree_predict(instance)`.

In [ ]:
def tree_predict_recu(node, x):
    """
    Return a prediction of an instance on the basis of the passed-in decision tree
    :param node: the current node that we're looking at
    :param x: the instance that we're predicting
    :return: the predicted label of the instance
    """
    # if we've reached a leaf, we have a prediction!
    if node.is_leaf():
        return node.leaf_label
    else:
        # take a look at where we should go down the tree given the values
        x_value = x[node.attribute]
        next_node = node.children[x_value]
        return tree_predict_recu(next_node, x)

    
def tree_predict(x):
    """
    Return a prediction of an instance on the basis of the passed-in decision tree.
    This function is meant to make the first call to the recursive function.
    :param x: the instance that we're predicting
    """
    return tree_predict_recu(decision_tree, x)

Simple enough! Now we can take a look at a random prediction!

In [ ]:
print_random_prediction(tree_predict, test_dicts)

Does that seem like a reasonable prediction? Would you have made the same prediction?

When we're working with decision trees, we can do something fun. We can traverse the tree as humans! We're going to look at the shortest review in our test set, and then you can manually traverse the tree answering the questions by taking a look at the article.

Let's write some code that does this.

In [ ]:
# let's print the shortest review in the test set.
shortest = min(test, key=len)
print(' '.join(shortest))

def manual_traverse(node):
    """
    Recursively lets you manually traverse a decision tree
    :param node: the node that we're currently looking at. Should initially be the root.
    """
    if node.is_leaf():
        print("I predict the review is %s." %node.leaf_label)
    else:
        ans = None
        while ans != 'Y' and ans != 'N':
            ans = input("Does the review have the word %s in it? " %node.attribute)
            if ans == 'Y':
                child = node.children[True]
                manual_traverse(child)
            elif ans == 'N':
                child = node.children[False]
                manual_traverse(child)
            else:
                print("Sorry, I don't understand what %s means.\nPlease make your answer Y or N.")

        
print("------------------\nPlease give answer as either Y or N.")
manual_traverse(decision_tree)

Hey, that was fun! Now let's evaluate our model. 

In [ ]:
evaluate(tree_predict, test_dicts)

Oof, yikes! I got 62.60%! That's really low! That's 14.7% **worse** than kNN! Oh no! Remember how I said that if your algorithm does worse than kNN, you know somethings wrong? Well, we now know something is wrong.

One interesting fact about decision trees is that smaller trees almost always score higher than bigger trees. Why? Because of a concept called *overfitting*. Overfitting is when a machine learning algorithm finds something that is true about the training set, assumes it to be true of all possible data, but it does not generalize to new data. For instance, say there's only one review with the word "vampire" in the training set, and that review is negative. Our algorithm might assume that all reviews containing that word are negative. However, after training, it might see an overwhelmingly positive review of *Nosferatu*. Our algorithm would be overfitted.

Smaller trees tend to be less overfitted, because they make less assumptions about the data. There are fewer branches, and thus fewer beliefs about the data. The remaining beliefs tend to more accurately capture the data in general.

So if we made our tree smaller, we might get a higher score. How could we make it smaller? There's a simple way to do it! Recall these lines from the training algorithm:

```
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
```

If there are no instances in the subset $S_a(v)$, then we make a leaf with the most common label. Although it's in our pseudocode and code, this actually should never happen in our case, because we only have two possible labels. If one of the $S_a(v)$s is empty, then the other subset will be equal to the set at the beginning, and thus the information gain should be 0, so it shouldn't be chosen as a best attribute.

We could make a modification to this line to 1) make it useful in our code and 2) make our tree smaller in a smart way. We could make it read like so:

```
            If the size of Examples(vi) is below a given threshold
                Then below this new branch add a leaf node with label = most common target value in the Examples(vi)
```

Now if $\vert S_a(v) \vert$ gets too small, it will just make a leaf with the most common label in that subset, assuming that this will be more accurate than making lots of assumptions about a very small amount of data. So we can pick a threshold below which it seems like it would be unreasonable to make generalizations from. Let's say 35.

We can rewrite our train function ever so slightly, like so:

In [ ]:
def id3_train(data, attributes):    
    """
    A recursive function which trains a decision tree based on the ID3 algorithm
    :param data: the data set we're currently looking at. This will get progressively smaller as we dichotomize.
    :param all_values: a dict mapping attributes to all their values. This will also get progressively smaller.
    :return: the trained tree
    """
    # let's create that root of the tree
    root = Node()
    # let's get a list of the labels we're working with
    labels = [point['LABEL'] for point in data]
    # if all instances are entirely one or the other label, we'll return a leaf node with that label
    if all(label == 'pos' for label in labels):
        root.leaf_label = 'pos'
        return root
    elif all(label == 'neg' for label in labels):
        root.leaf_label = 'neg'
        return root
    # we need to get the most common label in the data, which will be useful in several situations
    most_common_label = Counter(labels).most_common(1)[0][0]
    if len(attributes) == 0:
        root.leaf_label == most_common_label
        return root
    # If none of the above basecases are true, let's work on the recursive case.
    else:
        # find the best attribute
        a = argmax_infogain(data)
        # use it to make a node
        node = Node(attribute=a)
        # make a deep copy of the dict of attributes and values and delete the attribute just used from the dict
        less_attributes = list(attributes)
        less_attributes.remove(a)
        # train on the remainder for each possible value of this node's attribute:
        # True for contains that word, False for does not contain
        for value in (True, False):
            S_av = subset(data, a, value)
            # (if there's no more data with that value, we'll just give that node the most common label)
            if len(S_av) <= 35:
                S_av_labels = [point['LABEL'] for point in S_av]
                most_common_S_av = Counter(S_av_labels).most_common(1)[0][0]
                node.children[value] = Node(leaf_label=most_common_S_av)
            else:
                node.children[value] = id3_train(S_av, less_attributes)
        return node
    
decision_tree = id3_train(train_dicts, all_words)

Now, does this do better than our previous, slightly different algorithm? Let's find out.

In [ ]:
evaluate(tree_predict, test_dicts)

Hey, I got 65.80%! That's better than the previous version of the algorithm by 5.11%! It's better, but it's clearly still much than Naive Bayes and even our simple kNN.

The fact is, decision trees are actually rarely used in Automatic Document Classification in industry. Decision trees do not perform well in situations where there are a ton of attributes. In our case we have around 2,000 attributes, and very few of them are hugely predictive. Our tree is struggling under all these attributes. So although decision trees are relatively easy for humans to understand, and are great for certain types of problems, they are not that good at this ADC.

So we haven't yet gotten near state-of-the-art results and methods, but hopefully you have learned something about machine learning and the math behind it! This is a work in progress, so if you check back later, I might have more some new methods for you. Thanks for reading and following along.